In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 데이터 로딩

In [ ]:
# CIFAR-10 로드 (채널-마지막: BHWC)
trainset_raw = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
testset_raw  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

In [ ]:
# numpy → tensor 변환 & 스케일링 + (옵션) 전처리 증강  # <-- CHANGED: 정규화 제거, 증강 추가
def preprocess_cifar(dataset, augment=False, n_aug=10):
    # dataset.data: (N, 32, 32, 3) uint8
    images = torch.from_numpy(dataset.data).permute(0, 3, 1, 2).contiguous().float() / 255.0  # -> (N, 3, 32, 32) in [0,1]
    labels = torch.tensor(dataset.targets, dtype=torch.long)

    if not augment:
        return images, labels

    # 전처리 단계에서 생성할 텐서 기반 증강 파이프라인 (정규화 없음, [0,1] 유지)
    aug = T.Compose([
        # 1) 임의 크롭 후 32x32로 리사이즈 (CIFAR-10은 원본이 32x32라 scale을 너무 작게 잡지 않음)
        #    - size: 최종 출력 크기
        #    - scale: 원본 면적 대비 크롭 영역 비율 범위(0.6~1.0 권장, 너무 작으면 정보 손실 큼)
        #    - ratio: 가로세로비 범위(3/4~4/3)
        #    - interpolation: 보간 방식(BILINEAR가 일반적으로 부드럽고 안정적)
        #    - antialias: 다운샘플링 시 앤티앨리어싱 사용(계단현상 감소)
        T.RandomResizedCrop(
            size=32,
            scale=(0.6, 1.0),
            ratio=(3/4, 4/3),
            interpolation=T.InterpolationMode.BILINEAR,
            antialias=True
        ),

        # 2) 수평 뒤집기
        #    - p: 적용 확률(0.5가 일반적인 기본값)
        T.RandomHorizontalFlip(p=0.5),

        # 3) 수직 뒤집기
        #    - p: 적용 확률(수직 뒤집기는 데이터에 따라 해가 될 수 있어 낮게 설정; 필요 시 0으로 꺼도 됨)
        T.RandomVerticalFlip(p=0.1),

        # 4) 회전
        #    - degrees: 회전 각도 범위
        #    - interpolation: 회전 시 보간 방식
        #    - expand: True면 회전 후 이미지가 잘리지 않도록 캔버스 확대(32x32 고정이 필요하므로 False)
        #    - center: 회전 중심(None이면 중앙)
        #    - fill: 회전으로 생기는 바깥 영역 채움값(0.0=검정; 텐서 입력에서 채널별로 브로드캐스트)
        T.RandomRotation(
            degrees=(-30, 30),
            interpolation=T.InterpolationMode.BILINEAR,
            expand=False,
            center=None,
            fill=0.0
        ),

        # 5) 패치 지우기(Random Erasing)
        #    - p: 적용 확률
        #    - scale: 지워질 패치의 면적 비율(전체 대비)
        #    - ratio: 패치의 종횡비 범위
        #    - value: 채움값('random'이면 [0,1] 범위 임의값으로 채움 → 정규화 안 하는 현재 설정과 잘 맞음)
        #    - inplace: 입력 텐서를 제자리(in-place) 수정할지 여부(False 권장)
        T.RandomErasing(
            p=0.25,
            scale=(0.02, 0.15),
            ratio=(0.3, 3.3),
            value='random',
            inplace=False
        ),

        # 6) 아핀 변환(이동/스케일/전단 포함)
        #    - degrees: 회전 각도 범위
        #    - translate: 가로/세로 이동 비율(0.1=이미지 크기의 10%)
        #    - scale: 스케일 변화 범위
        #    - shear: 전단(기울이기) 각도 범위(단일 값/튜플 모두 허용, 여기선 좌우 대칭 범위로 지정)
        #    - interpolation: 보간 방식
        #    - center: 변환 기준점(None=중앙)
        #    - fill: 변환으로 생기는 바깥 영역 채움값
        T.RandomAffine(
            degrees=15,
            translate=(0.1, 0.1),
            scale=(0.9, 1.1),
            shear=(-10, 10),
            interpolation=T.InterpolationMode.BILINEAR,
            center=None,
            fill=0.0
        )
    ])

    N = images.shape[0]
    aug_imgs = []
    # 라벨은 나중에 한 번에 반복
    for i in range(N):
        x = images[i]
        for _ in range(n_aug):
            y = aug(x)
            # 혹시 색/보간으로 경계값 벗어나면 [0,1]로 클램프
            y = torch.clamp(y, 0.0, 1.0)
            aug_imgs.append(y)
    images = torch.stack(aug_imgs, dim=0)  # (N*n_aug, 3, 32, 32)
    labels = labels.unsqueeze(1).repeat(1, n_aug).reshape(-1)  # (N*n_aug,)
    return images, labels

In [ ]:
# 학습용은 증강, 테스트는 증강 없음
train_images, train_labels = preprocess_cifar(trainset_raw, augment=True, n_aug=5)
test_images,  test_labels  = preprocess_cifar(testset_raw,  augment=False)

In [ ]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

In [ ]:
# Dataset 객체로 묶기
full_dataset = torch.utils.data.TensorDataset(train_images, train_labels)

In [ ]:
# train/val split (Dataset을 그대로 분할)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

# 모델 생성

In [ ]:
# !pip install torchinfo torchview

In [ ]:
from torchinfo import summary

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class ResidualBlock(nn.Module):
    """
    가장 기본적인 Residual 블록
    - 입력 x와, Conv(3x3)-BN-ReLU-Conv(3x3)-BN 결과를 더한 뒤(ReLU 이전), 최종 ReLU 적용
    - 채널 수/공간 크기가 동일할 때만 사용 (여기서는 32채널 구간, 64채널 구간에 사용)
    """
    def __init__(self, channels: int):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(channels)
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(channels)

    def forward(self, x):
        identity = x                           # 잔차 경로(그대로 더하기 위한 경로)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = out + identity                   # 입력과 합치기(skip connection)
        out = self.relu(out)                   # 합친 뒤 활성화
        return out

In [ ]:
class CNN(nn.Module):
    """
    입력: (B, 3, 32, 32)
    출력: (B, 10)  # CrossEntropyLoss에 바로 사용
    - 변경점: 각 스테이지에서 간단한 ResidualBlock을 1개씩 추가
    """
    def __init__(self, num_classes: int = 10, p_drop: float = 0.25):
        super().__init__()

        self.features = nn.Sequential(
            # ----- Stage 1: 3 -> 32 채널 -----
            nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False),  # 32x32 -> 32x32
            nn.BatchNorm2d(32),
            nn.ReLU(),
            ResidualBlock(32),                                       # (간단 Residual: 32채널 유지)
            nn.MaxPool2d(2),                                         # 32x32 -> 16x16

            # ----- Stage 2: 32 -> 64 채널 -----
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False), # 16x16 -> 16x16
            nn.BatchNorm2d(64),
            nn.ReLU(),
            ResidualBlock(64),                                       # (간단 Residual: 64채널 유지)
            nn.MaxPool2d(2),                                         # 16x16 -> 8x8
        )

        self.avgpool = nn.AdaptiveAvgPool2d(1)  # -> (B, 64, 1, 1)
        self.classifier = nn.Sequential(
            nn.Flatten(),                    # -> (B, 64)
            nn.Dropout(p_drop),
            nn.Linear(64, num_classes),      # -> (B, 10)
        )

    def forward(self, x):
        x = self.features(x)                 # 특징 추출
        x = self.avgpool(x)                  # 글로벌 평균 풀링(채널만 남김)
        x = self.classifier(x)               # 분류기
        return x

In [ ]:
model = CNN().to(device)  # 모델을 GPU로 이동
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
summary(model, input_size=(1, 3, 28, 28))

In [ ]:
from torchview import draw_graph

In [ ]:
draw_graph(model, input_size=(1, 3, 28, 28)).visual_graph

# 학습

In [ ]:
# 로그 저장용 리스트
train_losses, train_accuracies, valid_losses, valid_accuracies = [], [], [], []

In [ ]:
for epoch in range(30):                       # 총 40번 데이터셋을 반복
    model.train()                             # 학습 모드(드롭아웃/배치정규화 등 학습 동작 켜짐)
    running_loss = 0                          # 에폭 동안의 학습 손실 누적 변수
    train_correct, train_total = 0, 0         # (추가) 에폭 단위 Train Acc 계산용

    # ---- 미니배치 학습 루프 ----
    for inputs, labels in trainloader:        # DataLoader에서 (입력, 정답) 배치를 꺼냄
        inputs, labels = inputs.to(device), labels.to(device)  # 배치 텐서를 모델과 같은 장치로 이동
        optimizer.zero_grad()                 # 직전 step의 누적 gradient를 0으로 초기화
        outputs = model(inputs)               # 순전파: 모델이 logits(미규격화 점수) 출력
        loss = loss_fn(outputs, labels)       # 손실 계산(CrossEntropyLoss: softmax+NLL 통합)
        loss.backward()                       # 역전파: 각 파라미터의 gradient 계산
        optimizer.step()                      # 가중치 갱신(gradient를 이용해 한 스텝 업데이트)
        running_loss += loss.item()           # 현재 배치의 손실 값을 파이썬 float로 누적

        # (추가) 배치 예측으로 Train Acc 누적
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_losses.append(running_loss / len(trainloader))  # 에폭 평균 학습 손실 기록 (배치 개수로 나눔)
    train_accuracies.append(100 * train_correct / train_total)     # 학습 정확도(%) 기록

    # ----- Validation -----
    model.eval()                              # 평가 모드(드롭아웃/BN 등 평가 동작)
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():                     # 평가 시에는 gradient 계산 비활성화(메모리/속도 이점)
        for inputs, labels in valloader:      # 검증 데이터 배치 반복
            inputs, labels = inputs.to(device), labels.to(device)  # 장치 정렬
            outputs = model(inputs)           # 순전파만 수행
            loss = loss_fn(outputs, labels)   # 검증 배치 손실
            val_loss += loss.item()           # 손실 누적
            _, predicted = torch.max(outputs, 1)     # 각 샘플의 최고 점수 클래스 인덱스
            total += labels.size(0)                  # 총 샘플 수 누적
            correct += (predicted == labels).sum().item()  # 맞춘 개수 누적

    valid_losses.append(val_loss / len(valloader))     # 에폭 평균 검증 손실 기록
    valid_accuracies.append(100 * correct / total)     # 검증 정확도(%) 기록

    # 진행 상황 출력
    print(f"Epoch {epoch+1}/50 : TRAIN[Loss: {train_losses[-1]:.4f}, Acc: {train_accuracies[-1]:.2f}%], VALID[Loss: {valid_losses[-1]:.4f}, Acc: {valid_accuracies[-1]:.2f}%]")

# 학습결과 확인

In [ ]:
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(valid_losses, label="Valid Loss")
plt.legend()
plt.title("Loss")
plt.show()

In [ ]:
plt.figure()
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(valid_accuracies, label="Valid Accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()